# Nesreče na smučiščih

Vsako leto se v smučarski sezoni zgodi veliko nesreč, zato bova poskušala podatke o preteklih nesrečah analizirati in ugotoviti, kateri so ključni razlogi za nastanek nesreče, kakšne so najpostejše posledice, kakšni so bili pogoji smučanja in podobno.

## Podatki

[Povezava](https://podatki.gov.si/dataset/evidenca-nesrec-na-smuciscih1)

Podatke sva našla na spletni strani podatki.gov.si. Ker so nama datoteke CSV bolj domače od PCAXIS sva se odločila za te, čeprav je to precej zmanjšalo najino izbiro. Datoteka obsega podrobne informacije o nesrečah na slovenskih smučiščih za leta od 2013 do vključno 2017.

V podatkih je 23 atributov, to so oznaka zadeve, različne okoliščine nesreče (ura, veter, vreme...), podatki o udeležencih in drugo. Med podatki je veliko napak in manjkajočih vrednosti, zato bo te težave potrebno najprej odpraviti.

## Uporabljene knjižnice

In [ ]:
import numpy as np
from csv import DictReader
from collections import OrderedDict
from matplotlib import pyplot as plt

## Branje podatkov

In [ ]:
# Datoteka mora biti v enaki mapi kot Main.py
# Vrne seznam slovarjev - vsaka vrstica je svoj slovar
def read_file(filename):
    try:
        with open(filename) as file:
            reader = DictReader(file, delimiter=";")
            return list(reader)
        
    except FileNotFoundError:
        print("File {0} not found. ".format(filename))

In [ ]:
# read data
data = read_file("podatki.csv")

## Atributi

Atributi so v datoteki shranjeni vsak z svojo zaporedno številko, začnejo se še z veliko začetnico in vsebujejo tudi šumnike in druge znake, zato sva najprej te pretvorila v utrezno obliko. Odstranila sva zaporedno številko atributa, ime pretvorila v male črke ter zamenjala presledke s podčrtaji.

Skupno je 23 atributov, ki so navedeni spodaj.

In [ ]:
# Odstrani šumnike in vodilne številke,
# pretvori v male črke
def format_key(attribute):
    slovar = {'č': 'c', 'š': 's', 'ž':'z', 'ć': 'c'}
    attribute = attribute.lower()
    splitted = attribute.split(" ")
    splitted.pop(0)
    output = ""
    for c in "_".join(splitted):
        if c in slovar:
            output += slovar[c]
        else: 
            output += c
    return output

In [ ]:
def format_attributes(rows):
    for i, row in enumerate(rows):
        output = {}
        for key, value in row.items():
            output[format_key(key)] = value
        rows[i] = output

In [ ]:
# formatiranje atributov
format_attributes(data)

# Izpis atributov
for key in data[0].keys():
    print(key)

## Filtriranje podatkov

Določenim podatkom manjkajo vrednosti, ali pa so po nepotrebnem dodane. Nekaterim letom na primer manjka vodilna števka (983 namesto 1983), nekateri pa imajo vrednosti dodane, npr. '4 s snegom povsem pokrito smučišče' in 's snegom povsem pokrito smučišče' sta dve vrsti snega, čeprav gre za isto stvar. Te in podobne težave je potrebno odpraviti. 

In [ ]:
def replace_value(value):
    value = value.strip(" ").lower()
    value = value.split(" ")
    value.pop(0)
    if value:
        return " ".join(value)
    return None

def replace_bool(value):
    value = value.strip()
    if not value or value == "Ne":
        return False
    return True

def replace_bool_unstrict(value):
    value = value.strip().lower()
    if not value:
        return None
    if value == "ne":
        return False
    return True

def replace_year(value):
    value = value.strip()
    if not value:
        return None
    if len(value) == 3:
        return "1" + str(value)
    return value

def replace_none(value):
    value = value.strip(" ").lower()
    if not value:
        return None
    return value

In [ ]:
for i, row in enumerate(data):
    
    # nadzornik_prisoten, zdravnik_prisoten, resevalec_prisoten
    data[i]["nadzornik_prisoten"] = replace_bool_unstrict(row["nadzornik_prisoten"])
    data[i]["zdravnik_prisoten"] = replace_bool_unstrict(row["zdravnik_prisoten"])
    data[i]["resevalec_prisoten"] = replace_bool_unstrict(row["resevalec_prisoten"])
    
    # udeleženec A in B leto rojstva
    data[i]["udelezenec_a_–_leto_rojstva"] = replace_year(row["udelezenec_a_–_leto_rojstva"])
    data[i]["udelezenec_b_–_leto_rojstva"] = replace_year(row["udelezenec_b_–_leto_rojstva"])
    
    # udeleženec A in B ravnanje pred nesrečo
    data[i]["udelezenec_a_–_ravnanje_pred_nesreco"] = replace_value(row["udelezenec_a_–_ravnanje_pred_nesreco"])
    data[i]["udelezenec_b_–_ravnanje_pred_nesreco"] = replace_value(row["udelezenec_b_–_ravnanje_pred_nesreco"])
    
    # udeleženec A in B država
    data[i]["udelezenec_a_–_drzava"] = replace_none(row["udelezenec_a_–_drzava"])
    data[i]["udelezenec_b_–_drzava"] = replace_none(row["udelezenec_b_–_drzava"])
    
    # skica nesreče
    data[i]["skica_nesrece"] = replace_bool(row["skica_nesrece"])
    
    # fotografija nesreče
    data[i]["fotografija_kraja_nesrece"] = replace_bool(row["fotografija_kraja_nesrece"])
    
    # posnetek kraja nesreče
    data[i]["posnetek_kraja_nesrece"] = replace_bool(row["posnetek_kraja_nesrece"])
    
    # vreme
    data[i]["vreme"] = replace_value(row["vreme"])
    
    # vidljivost
    data[i]["vidljivost"] = replace_value(row["vidljivost"])
       
    # vrsta snega
    data[i]["vrsta_snega"] = replace_value(row["vrsta_snega"])
    
    # veter
    data[i]["veter"] = replace_value(row["veter"])

## Nesreče po posamezni uri

In [ ]:
# Parameter rows so prebrani podatki
def accident_times(rows):
    hours = {}
    for row in rows:
        time = row["ura_nesrece"]
        hour = time.split(":")[0]
        if hour not in hours:
            hours[hour] = 0
        hours[hour] += 1

    # sort by hour
    count = OrderedDict(sorted(hours.items(), reverse=True))

    x = list(count.keys())
    y = list(count.values())
    plt.barh(x, y)
    plt.title("Nesreče po urah")
    plt.xlabel("Št. nesreč")
    plt.ylabel("Ura")
    plt.show()

In [ ]:
# plot accidents at given hour
accident_times(data)

Vidi se, da se največ nesreč zgodi med deveto uro zjutraj in štirih popoldne, ko je na smučišču največ ljudi. Med 22. uro in polnočjo pa ni bilo zabeležene nobene nesreče.

## Nesreče po mescih

Za vsako nesrečo je zabeležen datum, kdaj je do nje prišlo. Ta datum sva v naslednjem primeru pretvorila v številko mesca in preštela število nesreč za posamezni mesec. <br>Kot pričakovano je največ nesreč ravno v zimskih mesecih, ko se novembra začne smučarska sezona in traja vse do aprila. V poletnih mescih nesreč skoraj da ni, avgusta, septembra in oktobra pa ni bila zabeležena nobena nesreča.

In [ ]:
# Nesreče glede na mesec
def accident_date(rows):
    dates = {}
    
    for row in rows:
        date = row["datum_nesrece"]
        month = date.split(".")[1]
        if month not in dates:
            dates[month] = 0
        dates[month] += 1
    
    # sort by month
    count = OrderedDict(sorted(dates.items()))
    
    x = list(count.keys())
    y = list(count.values())
    plt.bar(x, y)
    plt.title("Nesreče po mescih")
    plt.ylabel("Št. nesreč")
    plt.xlabel("Mesec")
    plt.show()
    
accident_date(data)

## Prisotno osebje

In [ ]:
def counter(rows, column):
    slovar = {}
    
    for row in rows:
        col = row[column]
        if col not in slovar:
            slovar[col] = 0
        slovar[col] += 1
    
    return slovar

In [ ]:
def present_personnel(rows):
    
    array = counter(rows, "nadzornik_prisoten")
    array = dict(sorted(array.items(), key=lambda t: (str(t[1]), str(t[0])), reverse=True))
    
    x1 = list(array.keys())
    y1 = list(array.values())
    
    array = counter(rows, "zdravnik_prisoten")
    array = dict(sorted(array.items(), key=lambda t: (str(t[1]), str(t[0])), reverse=True))
    
    x2 = list(array.keys())
    y2 = list(array.values())
    
    values = [y1, y2]

    X = np.arange(3)
    plt.bar(X + 0.00, values[0], color = 'r', width = 0.33)
    plt.bar(X + 0.33, values[1], color = 'b', width = 0.33)

    plt.show()

present_personnel(data)